In [1]:
import matplotlib.pyplot as plt
import numpy as np
import re
import os
import pandas as pd
import ast

plt.rcParams.update({
    'font.size': 18,
    'text.color': 'black',
    'axes.labelcolor': 'white',
    'xtick.color': 'white',
    'ytick.color': 'white'
})
plt.figure(figsize=(20, 10))

data = {}

def extract_file_data(file):
    with open(file) as file:
        raw = file.read()
        failed = re.compile('Round \d+ failed clients: (\[[^]]*])').findall(raw)
        failed = list(map(len ,map(ast.literal_eval, failed)))
        avg_failed = np.mean(failed)
        rounds_failed = len(list(filter(lambda x: x > 0, failed)))
        selected = re.compile('Sampled clients for round \d+: (\[[^]]*])').findall(raw)
        selected = list(map(ast.literal_eval, selected))
        unique_participants = len(set().union(*selected))
        return avg_failed, unique_participants, rounds_failed

for dir in os.listdir('results'):
    config = ''
    with open('results/' + dir + '/args.txt') as args:
        args = args.read()
        config = re.compile('trace_distro = (\S+)').search(args).group(1)
    accuracies = np.load('results/' + dir + '/accuracies.npy')
    for file in os.listdir('results/' + dir):
        if file.startswith('run.sh'):
            avg_failed, unique_participants, rounds_failed = extract_file_data('results/' + dir + '/' + file)

    plt.plot(range(len(accuracies)), accuracies, label=config)
    times = np.load('results/' + dir + '/times.npy')
    data[config] = {
        'finish_time': times[-1],
        'mean_acc': accuracies[1500:].mean(),
        'std_acc': accuracies[1500:].std(),
        'avg_failed': avg_failed,
        'rounds_failed': rounds_failed,
        'unique_participants': unique_participants
    }




plt.xlabel('Round number')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

pd.DataFrame(data)

KeyboardInterrupt: 

<Figure size 1440x720 with 0 Axes>